# 📂 Подготовка структуры проекта в Google Colab

В этом блоке я:

- Клонирую репозиторий с GitHub в рабочее пространство Google Colab, чтобы все изменения и новый код автоматически сохранялись в облачном репозитории.
- Перехожу в директорию проекта (все храню на google диске), чтобы всё, что буду создавать дальше, находилось в одной организованной папке.
- Создаю стандартную структуру каталогов для удобства работы над проектом:
  - `src/etl` — для скриптов по сбору и обработке данных (ETL: Extract, Transform, Load)
  - `data/raw` — для хранения исходных (сырых) данных, полученных из hh.ru
  - `models` — для хранения файлов обученных моделей машинного обучения
  - `notebooks` — для Jupyter/Colab-ноутбуков, в которых буду проводить анализ, эксперименты и демонстрацию результатов



In [ ]:
!git clone https://github.com/Annut04ka/hh-hr-bot.git /content/drive/MyDrive/hh-hr-bot
%cd /content/drive/MyDrive/hh-hr-bot
!mkdir -p src/etl data/raw models notebooks

Cloning into '/content/drive/MyDrive/hh-hr-bot'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 6 (delta 0), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (6/6), done.
/content/drive/MyDrive/hh-hr-bot


# 📝 В этой части кода я создаю файл requirements.txt

Здесь формируется стандартный список всех библиотек Python, необходимых для запуска и работы нашего проекта.

- Такой файл нужен для воспроизводимости: любой человек или сервис, скачавший наш проект с GitHub, сможет одной командой установить все нужные зависимости.
- Это важно для быстрой настройки окружения в Google Colab, на локальном компьютере, или при автоматическом развёртывании на сервере.

Все библиотеки перечисляются по одной в строке — теперь нам не нужно вспоминать, что ставить, а проверяющий или коллега сможет легко воспроизвести нашу среду!


In [ ]:
%%writefile requirements.txt
pandas
duckdb
lxml
beautifulsoup4
requests
tqdm
python-dotenv

Writing requirements.txt


In [ ]:
# УДАЛИТЬ
!git add requirements.txt
!git commit -m "init: requirements"
!git push origin main

In [2]:
%cd /content/drive/MyDrive/hh-hr-bot

/content/drive/MyDrive/hh-hr-bot


In [ ]:
# В этой части кода мы создаём файл .env.example — пример для конфигурации проекта.
# Реальные значения (токены и пароли) НЕ публикуем, только шаблон!
%%writefile .env.example
TELEGRAM_TOKEN=your_telegram_token_here
SUPABASE_URL=https://xyzcompany.supabase.co
SUPABASE_KEY=your_supabase_key_here

Overwriting .env.example


In [ ]:
# В этой части кода мы устанавливаем все необходимые библиотеки,
# чтобы Python-код мог их импортировать и выполнять.
!pip install duckdb lxml beautifulsoup4 requests tqdm python-dotenv

In [ ]:
# В этой части кода мы переходим в папку проекта на Google Диске,
# чтобы все файлы сохранялись именно там и не терялись после отключения Colab.
%cd /content/drive/MyDrive/hh-hr-bot

/content/drive/MyDrive/hh-hr-bot


In [17]:
# В этой части кода мы собираем вакансии через API hh.ru:
# сначала выгружаем id, потом для каждой вакансии берём подробную информацию
# (description, key_skills и др.), и сохраняем результат в csv и DuckDB.
%%writefile src/etl/fetch_hh.py
import requests
import time
import pandas as pd
import duckdb
from tqdm import tqdm
from pathlib import Path

Path("data/raw").mkdir(parents=True, exist_ok=True)

# Какие регионы выгружаем (можно расширить!)
REGIONS = [
    1,    # Москва
    2,    # Санкт-Петербург
    3,    # Екатеринбург
    4,    # Новосибирск
    66,   # Красноярск
    78,   # Нижний Новгород
    # Добавь ещё, если хочешь (id смотри в справочнике hh.ru)
]

PAGES_PER_REGION = 5   # максимум 500 вакансий на регион
PER_PAGE = 100

def fetch_vacancy_ids(pages=5, per_page=100, area="1"):
    """Собирает id вакансий по поиску"""
    vac_ids = []
    for page in tqdm(range(pages), desc="Выгружаем id"):
        url = "https://api.hh.ru/vacancies"
        params = dict(area=area, per_page=per_page, page=page, text="*")
        r = requests.get(url, params=params)
        for item in r.json()["items"]:
            vac_ids.append(item["id"])
        time.sleep(0.2)
    return vac_ids

def fetch_full_vacancies(vac_ids):
    """Детально собирает вакансии по id (description, key_skills и др.)"""
    all_rows = []
    for vid in tqdm(vac_ids, desc="Грузим вакансии по id"):
        url = f"https://api.hh.ru/vacancies/{vid}"
        v = requests.get(url).json()
        all_rows.append({
            "id": v["id"],
            "title": v["name"],
            "published_at": v["published_at"],
            "description": v.get("description", ""),
            "salary_from": (v["salary"] or {}).get("from") if v.get("salary") else None,
            "salary_to": (v["salary"] or {}).get("to") if v.get("salary") else None,
            "salary_currency": (v["salary"] or {}).get("currency") if v.get("salary") else None,
            "experience_hh": (v.get("experience") or {}).get("name"),
            "area_id": int(v["area"]["id"]),
            "skills_raw": ", ".join(s["name"] for s in v.get("key_skills", []))
        })
        time.sleep(0.05)  # не перегружаем hh.ru
    return pd.DataFrame(all_rows)

if __name__ == "__main__":
    # 5 страницы по 100 = 500 вакансий на регион (можно увеличить до лимита)
    #vac_ids = fetch_vacancy_ids(pages=3, per_page=100, area="113")
    #df = fetch_full_vacancies(vac_ids)
    all_vac_ids = set()
    for area in REGIONS:
        print(f"\n=== Грузим регион area_id={area} ===")
        ids = fetch_vacancy_ids(pages=PAGES_PER_REGION, per_page=PER_PAGE, area=str(area))
        all_vac_ids.update(ids)
    print(f"\nИтого уникальных вакансий: {len(all_vac_ids)}")

    df = fetch_full_vacancies(list(all_vac_ids))
    # Сохраняем в csv
    df.to_csv("data/raw/sample_vacancies_3000.csv", index=False)
    # Сохраняем в DuckDB
    con = duckdb.connect("data/hh.duckdb_3000")
    con.execute("CREATE OR REPLACE TABLE vacancy AS SELECT * FROM df")
    con.close()
    print(f"Сохранили {len(df)} вакансий в data/raw/sample_vacancies.csv и data/hh.duckdb")

Overwriting src/etl/fetch_hh.py


In [19]:
# В этой части кода мы запускаем наш скрипт для выгрузки вакансий через API hh.ru.
!python src/etl/fetch_hh.py


=== Грузим регион area_id=1 ===
Выгружаем id: 100% 5/5 [00:09<00:00,  1.89s/it]

=== Грузим регион area_id=2 ===
Выгружаем id: 100% 5/5 [00:09<00:00,  1.80s/it]

=== Грузим регион area_id=3 ===
Выгружаем id: 100% 5/5 [00:09<00:00,  1.84s/it]

=== Грузим регион area_id=4 ===
Выгружаем id: 100% 5/5 [00:09<00:00,  1.90s/it]

=== Грузим регион area_id=66 ===
Выгружаем id: 100% 5/5 [00:09<00:00,  1.84s/it]

=== Грузим регион area_id=78 ===
Выгружаем id: 100% 5/5 [00:09<00:00,  1.81s/it]

Итого уникальных вакансий: 2995
Грузим вакансии по id: 100% 2995/2995 [50:46<00:00,  1.02s/it]
Сохранили 2995 вакансий в data/raw/sample_vacancies.csv и data/hh.duckdb


In [18]:
import requests
import pandas as pd

# Получаем справочник регионов hh.ru (Россия — area_id = 113, но лучше выгрузить все)
areas_url = "https://api.hh.ru/areas"
resp = requests.get(areas_url)
areas_json = resp.json()

# Соберём в DataFrame плоский список: id и name для каждого региона и города
def parse_areas(areas, parent=None):
    rows = []
    for area in areas:
        rows.append({
            "area_id": int(area['id']),
            "area_name": area['name'],
            "parent_area": parent
        })
        # Рекурсия по подрегионам и городам
        if area.get('areas'):
            rows.extend(parse_areas(area['areas'], parent=area['name']))
    return rows

areas_list = parse_areas(areas_json)
areas_df = pd.DataFrame(areas_list)
areas_df.head()

,area_id,area_name,parent_area
0,113,Россия,None
1,1620,Республика Марий Эл,Россия
2,4228,Виловатово,Республика Марий Эл
3,1621,Волжск,Республика Марий Эл
4,1622,Звенигово,Республика Марий Эл


In [ ]:
areas_df.to_csv("data/areas_hh.csv", index=False)

In [3]:
# В этой части кода мы читаем выгруженный CSV-файл и смотрим, как выглядят наши данные.
import pandas as pd
df = pd.read_csv('data/raw/sample_vacancies_3000.csv')
df.head()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2995 entries, 0 to 2994
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               2995 non-null   int64  
 1   title            2995 non-null   object 
 2   published_at     2995 non-null   object 
 3   description      2995 non-null   object 
 4   salary_from      2227 non-null   float64
 5   salary_to        1286 non-null   float64
 6   salary_currency  2373 non-null   object 
 7   experience_hh    2995 non-null   object 
 8   area_id          2995 non-null   int64  
 9   skills_raw       1709 non-null   object 
dtypes: float64(2), int64(2), object(6)
memory usage: 234.1+ KB


In [4]:
# В этой части кода мы создаём таблицу навыков и связь между вакансиями и навыками.
import duckdb

con = duckdb.connect("data/hh.duckdb_3000")

# Используем только name как ключ (id не нужен для справочника навыков)
con.execute("CREATE TABLE IF NOT EXISTS skill(name TEXT PRIMARY KEY)")

skills = con.execute("SELECT skills_raw FROM vacancy").fetch_df()
uniq_skills = sorted({s.strip() for row in skills.skills_raw for s in str(row).split(",") if s and s.strip()})

for n in uniq_skills:
    con.execute("INSERT INTO skill(name) VALUES (?) ON CONFLICT DO NOTHING", [n])

# Создаём связь vacancy_skill (многие-ко-многим)
# con.execute("DROP TABLE IF EXISTS vacancy_skill")
con.execute("CREATE TABLE IF NOT EXISTS vacancy_skill(vacancy_id BIGINT, skill_name TEXT, PRIMARY KEY(vacancy_id, skill_name))")

for row in con.execute("SELECT id, skills_raw FROM vacancy").fetchall():
    vid, raw = row
    try:
        vid = int(vid)
    except Exception:
        print("⚠️ Некорректный id:", vid, "RAW:", raw)
        continue
    for sk in [s.strip() for s in str(raw).split(",") if s and s.strip()]:
        try:
            con.execute("INSERT OR IGNORE INTO vacancy_skill VALUES (?, ?)", [vid, sk])
        except Exception as e:
            print(f"Ошибка при вставке: {vid}, {sk}, {e}")

con.commit(); con.close()

In [9]:
# В этой части кода мы проводим первичный анализ (EDA) по базе вакансий и навыков.

import duckdb

# Подключаемся к базе данных
con = duckdb.connect("data/hh.duckdb_3000")

# 1. Сколько всего вакансий в базе?
num_vacancies = con.execute("SELECT COUNT(*) FROM vacancy").fetchone()[0]
print(f"Всего вакансий: {num_vacancies}")

# 2. Сколько уникальных навыков в справочнике?
num_skills = con.execute("SELECT COUNT(*) FROM skill").fetchone()[0]
print(f"Всего уникальных навыков: {num_skills}")

# 3. ТОП-10 самых востребованных навыков (по числу вакансий)
print("\nТОП-10 востребованных навыков:")
top_skills = con.execute("""
    SELECT skill_name, COUNT(*) AS n
    FROM vacancy_skill
    GROUP BY skill_name
    ORDER BY n DESC
    LIMIT 10
""").fetchdf()
print(top_skills)

# 4. Распределение вакансий по городам (по area_id)
print("\nРаспределение вакансий по городам (area_id):")
city_dist = con.execute("""
    SELECT area_id, COUNT(*) as n
    FROM vacancy
    GROUP BY area_id
    ORDER BY n DESC
    LIMIT 10
""").fetchdf()
print(city_dist)

con.close()


Всего вакансий: 2995
Всего уникальных навыков: 2464

ТОП-10 востребованных навыков:
                            skill_name    n
0                     Работа в команде  161
1                    Деловая переписка  152
2                      Деловое общение  145
3               Организаторские навыки  142
4                      Ответственность  133
5                      Пользователь ПК  124
6                       Грамотная речь  109
7                Телефонные переговоры  105
8  Работа с большим объемом информации  105
9                     Активные продажи   83

Распределение вакансий по городам (area_id):
   area_id    n
0        1  504
1       66  498
2        4  497
3        2  496
4       78  494
5        3  493
6       99    2
7      102    1
8       70    1
9       22    1


In [ ]:
%cd /content/drive/MyDrive/hh-hr-bot
!git pull origin main


/content/drive/MyDrive/hh-hr-bot
remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 3 (delta 1), reused 0 (delta 0), pack-reused 0 (from 0)
Unpacking objects: 100% (3/3), 1.85 KiB | 55.00 KiB/s, done.
From https://github.com/Annut04ka/hh-hr-bot
 * branch            main       -> FETCH_HEAD
   3c736c0..0f92aa4  main       -> origin/main
Updating 3c736c0..0f92aa4
Fast-forward
 README.md | 34 +++++++++++++++++++++++++++++++++-
 1 file changed, 33 insertions(+), 1 deletion(-)
